In [1]:
from transformers import AutoTokenizer,AutoModelForQuestionAnswering, TrainingArguments, Trainer,AutoConfig
from transformers import DefaultDataCollator
from transformers import TrainingArguments
from transformers import HfArgumentParser
from transformers import Trainer
from datasets import load_dataset
import torch
from transformers import DistilBertModel
from datasets import load_dataset
from transformers import PreTrainedModel,PretrainedConfig
from transformers.modeling_outputs import QuestionAnsweringModelOutput

In [26]:
# # load_dataset('/home/vp.shivasan/interiit/data/train_data.csv')
# my_dataset = load_dataset("csv", data_files="/home/vp.shivasan/interiit/data/train_data.csv",split="train[:5000]")
# # my_dataset.train_test_split(test_size=0.2)
# my_dataset[0]

In [27]:
from datasets import load_dataset
squad = load_dataset("squad", split="train[:5000]")
squad = squad.train_test_split(test_size=0.2)

Found cached dataset squad (/home/vp.shivasan/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
Loading cached split indices for dataset at /home/vp.shivasan/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-a75de2358c778f97.arrow and /home/vp.shivasan/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-8fb749f4a3130476.arrow


In [28]:
my_dataset = squad

In [29]:
# proportions = [.75, .10, .15]
# lengths = [int(p * len(my_dataset)) for p in proportions]
# train_set,val_set,test_set = torch.utils.data.random_split(my_dataset, lengths,generator=torch.Generator().manual_seed(42))

In [30]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

loading configuration file config.json from cache at /home/vp.shivasan/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /home/vp.shivasan/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /home/vp.shivasan/.cache/huggingface/hub/mo

In [31]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [32]:
tokenized_data = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Loading cached processed dataset at /home/vp.shivasan/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-e513ca363bbc9394.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

In [33]:
class DistillBERTQA(PreTrainedModel):
    def __init__(self,config: PretrainedConfig):
        # super(DistillBERTQA, selfconfig).__init__()
        super().__init__(config)
        self.distilbert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.qa_outputs = torch.nn.Linear(768, 2)
        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask,start_positions=None,end_positions=None,return_dict=None):
        distilbert_output = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = distilbert_output[0]
        hidden_states = self.dropout(hidden_states)
        logits = self.qa_outputs(hidden_states)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1).contiguous()  # (bs, max_query_len)
        end_logits = end_logits.squeeze(-1).contiguous()  # (bs, max_query_len)

        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)
            # sometimes the start/end positions are outside our model inputs, we ignore these terms
            ignored_index = start_logits.size(1)
            start_positions = start_positions.clamp(0, ignored_index)
            end_positions = end_positions.clamp(0, ignored_index)

            loss_fct = torch.nn.CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            total_loss = (start_loss + end_loss) / 2

        if not return_dict:
            output = (start_logits, end_logits) + distilbert_output[1:]
            return ((total_loss,) + output) if total_loss is not None else output

        return QuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            hidden_states=distilbert_output.hidden_states,
            attentions=distilbert_output.attentions,
        )


In [34]:
config = AutoConfig.from_pretrained("distilbert-base-uncased")

model = DistillBERTQA(config)


loading configuration file config.json from cache at /home/vp.shivasan/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading configuration file config.json from cache at /home/vp.shivasan/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architec

In [36]:
data_collator = DefaultDataCollator()

LOG_NAME = "task2_test"
args_dict = {
        "output_dir": '/home/vp.shivasan/interiit/train_dir/' + LOG_NAME,
        "overwrite_output_dir": True,
        "per_device_train_batch_size": 32, ## <<-- Change this if as per your GPU memory
        "per_device_eval_batch_size": 32,
        "learning_rate": 1e-3, 
        "warmup_steps": 250,
        "logging_steps": 10,
        "evaluation_strategy": "epoch",
        "eval_steps": 100,
        "num_train_epochs": 20,
        "do_train": True,
        "do_eval": True,
        "fp16": False,
        # "max_steps": 7200,
        # 'save_steps':200,
        'save_strategy':'epoch', 
        'load_best_model_at_end': True,
        'metric_for_best_model':'eval_loss',
        'greater_is_better':False,
        'seed':40,
        'data_seed':40,
        'local_rank':-1
    }
parser = HfArgumentParser(
        (TrainingArguments))
training_args = parser.parse_dict(args_dict)


training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model() 
trainer.save_state()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 4000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 63
  Number of trainable parameters = 66364418


Epoch,Training Loss,Validation Loss
1,No log,3.613123


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to my_awesome_qa_model
Configuration saved in my_awesome_qa_model/config.json
Model weights saved in my_awesome_qa_model/pytorch_model.bin
tokenizer config file saved in my_awesome_qa_model/tokenizer_config.json
Special tokens file saved in my_awesome_qa_model/special_tokens_map.json


In [37]:
preds,labels,metrics = trainer.predict(tokenized_data['test'])

***** Running Prediction *****
  Num examples = 1000
  Batch size = 64


In [43]:
preds[0].shape

(1000, 384)